# Model Evaluation on Test Data
This notebook evaluates the trained model on the test dataset. It displays each image, the actual label (disease), and the predicted label.

In [1]:
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import os
from train_pipeline import ModifiedResNet18  # Importing the ModifiedResNet18 class

# Load the trained model
model = ModifiedResNet18(num_classes=8)  # Match the saved model's configuration
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load test data
test_csv = "test.csv"
image_dirs = ["For Lee/pageOne/", "For Lee/pageTwo/", "For Lee/pageThree/"]
test_data = pd.read_csv(test_csv)

# Debugging: Check the size of the full test dataset
print(f"Full test data size: {len(test_data)}")

# Helper function to find image path
def find_image_path(filename, image_dirs):
    for image_dir in image_dirs:
        img_path = os.path.join(image_dir, filename)
        if os.path.exists(img_path):
            return img_path
    raise FileNotFoundError(f"Image {filename} not found in any of the directories.")

# Add loss calculation during evaluation
criterion = nn.CrossEntropyLoss()

# Debugging: Check the filenames being processed
for idx, row in test_data.iterrows():
    filename = row['filename']
    print(f"Processing file: {filename}")
    actual_label = row['disease']
    img_path = find_image_path(filename, image_dirs)

    # Load and preprocess the image
    image = Image.open(img_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0)

    # Predict
    with torch.no_grad():
        outputs = model(input_tensor)
        predicted_idx = torch.argmax(outputs, dim=1)
        predicted_label = ['healthy', 'sb'][predicted_idx.item()]

        # Calculate loss
        actual_idx = ['healthy', 'sb'].index(actual_label)
        loss = criterion(outputs, torch.tensor([actual_idx]))

    # Debugging: Print predictions and loss
    print(f"Actual: {actual_label}, Predicted: {predicted_label}, Loss: {loss.item()}")

    # Display the image and predictions
    plt.imshow(image)
    plt.title(f'Actual: {actual_label} | Predicted: {predicted_label} | Loss: {loss.item():.4f}')
    plt.axis('off')
    plt.show()

Epoch 1/10, Training Loss: 1.9248125553131104, Validation Loss: 1.5181788802146912
Epoch 2/10, Training Loss: 1.130476450920105, Validation Loss: 0.8343719840049744


KeyboardInterrupt: 